# 遗传算法python示例——背包问题

**问题描述**

现有一个背包，其容量上限为10，有4个物品可以放进该背包中，每个物品有不同的价值，求在背包容量范围内如何放置物品使得总价值最高。

物品价值：values= [10, 40, 30, 50]

物品重量：wt = [5, 4, 6, 3]

* 遗传算法基本流程：

![遗传算法基本流程](https://tva1.sinaimg.cn/large/007S8ZIlly1ghemckuex2j30jo0f8jry.jpg)

In [1]:
import numpy as np
import pandas as pd
import random
import itertools
import time

## 个体/染色体/基因

* 遗传算法中，个体即为解，染色体即为解的形式，基因则为解的组成部分。
* 在自然界中，1个个体内包含多条染色体，但在算法中，一般1个个体对应1个染色体，所以个体/染色体在算法中可理解为一个概念，即问题的解。
* 在本例中，共有4个物品，每个物品放入即为1，不放入即为1（共4个基因，每个基因为0/1，代表对应物体是否放置进背包）。解（个体/染色体）即为4位的二进制列。

In [2]:
class Individual(object):
    def __init__(self,evaluator,fixed_chromosome=[]):
        self.evaluator = evaluator
        self.gene_num = evaluator.gene_num # 基因数量
        self.chromosome = self.get_chromosome(fixed_chromosome)  # 染色体       
        
    def get_chromosome(self,fixed_chromosome=[]):
        if fixed_chromosome:
            chromosome = np.array(fixed_chromosome)
        else:
            chromosome = self.encode_chromosome()
        while self.evaluator.if_valid_chromosome(chromosome):  # 判断染色体是否合法
            chromosome = np.random.randint(0,2,self.gene_num)
        return chromosome

    def encode_chromosome(self):
        return np.random.randint(0,2,self.gene_num)

In [17]:
class Evaluator(object):
    def __init__(self,gene_num,values,weights, weight_MAX):  
        self.gene_num = gene_num  # 染色体中基因的数量，在本问题中即为物品的个数
        self.values = values  # 各物品对应的价值
        self.weights = weights  # 各物体所占的容量
        self.weight_MAX = weight_MAX  # 背包的容量上限
    
    def get_fitness_value(self, individual):
        individual.fitness_value = sum(individual.chromosome*self.values)
        return individual.fitness_value
    
    def if_valid_chromosome(self,chromosome):
        return sum(chromosome*self.weights) > self.weight_MAX

## 种群

In [28]:
class Population(object):
    def __init__(self, population_size, evaluator, fixed_chromosome=[]):
        self.population_size = population_size  # 种群中个体数量
        self.evaluator = evaluator  # 评价类
        self.individuals = []  # 种群内的个体集合
        for n in range(population_size):
            self.individuals.append(Individual(self.evaluator,fixed_chromosome=[]))
            
        self.best_individual_list = []  # 记录每一代中最好的个体
        self.best_individual_dict = {}  # 记录每一代中最好的个体及其适应度
        
    def evolve(self,offspring):
        fv_now_gen = []  # 当前代的种群中个体的适应度列表
        for individual in self.individuals:
            fv_now_gen.append(self.evaluator.get_fitness_value(individual))
        
        best_index = fv_now_gen.index(max(fv_now_gen))
        self.best_individual_list.append(self.individuals[best_index])  # 记录当前代中的最优个体
        self.best_individual_dict[self.individuals[best_index]] = max(fv_now_gen)  # 记录当前代中最优个体及其适应度

        # 加入子代进行淘汰，维持种群内个体个数
        fv_offspring=[]  # 子代的种群中个体的适应度列表
        for individual in offspring:
            fv_offspring.append(self.evaluator.get_fitness_value(individual))
        
        # 本例中种群内个体个数为100，则对适应度进行排序后，查找第100个适应度的值作为分界值，
        # 筛选适应度大于等于分界线的个体
        fv_all = fv_now_gen+fv_offspring  # 父代+子代的适应度列表
        fv_all.sort(reverse=True)
        cut_value = fv_all[self.population_size-1]  # 查找分界值
        
        new_population = []
        for individual in self.individuals:
            if self.evaluator.get_fitness_value(individual) >= cut_value:
                new_population.append(individual)
        for individual in offspring:
            if self.evaluator.get_fitness_value(individual) >= cut_value:
                new_population.append(individual)
                
        self.individuals = new_population  # 更新种群内的个体，进入下一代


In [29]:
class SelectionOperator(object):
    def __init__(self,population,parent_size):
        self.population = population  # 种群
        self.parent_size = parent_size  # 进入繁殖的父代个体数
        
    def roulette_wheel(self):  # 轮盘赌算法
        fitness_value_list=[]
        for individual in self.population.individuals:
            fitness_value_list.append(self.population.evaluator.get_fitness_value(individual))
        
        # 计算累计概率
        cumulate_p_list = []
        pre_p = 0
        for fitness_value in fitness_value_list:
            p=float(fitness_value/sum(fitness_value_list))
            cumulate_p_list.append(p+pre_p)
            pre_p = p+pre_p
        
        # 轮盘赌算法，适应度越大的个体越容易被选做可以繁殖的父代
        self.parents = []
        for n in range(self.parent_size):
            r = random.uniform(0,1)
            parents_pair=[]
            #print(n,'*'*10)
            for parent_num in range(2):  # 2个父代产生1个子代
                for i in range(len(cumulate_p_list)):
                    if i == 0:
                        if r <= cumulate_p_list[i]:
                            parents_pair.append(self.population.individuals[i])
                    else:
                        if r <= cumulate_p_list[i] and r > cumulate_p_list[i-1]:
                            parents_pair.append(self.population.individuals[i])
            self.parents.append(tuple(parents_pair))

        return self.parents
        

In [30]:
class VariationOperator(object):
    def __init__(self,evaluator,parents,p_point_mutation):
        self.evaluator = evaluator  # 评价类
        self.parents = parents  # 父代
        self.p_point_mutation = p_point_mutation  # 点突变概率
        self.offspring= []
        
    def get_offspring(self):
        self.crossover()  # 父代染色体交叉产生子代染色体
        self.point_mutation()  # 点突变
        return self.offspring
    
    def crossover(self):
        for pair in parents:
            cross_point = random.randint(0, evaluator.gene_num-1)  # 随机选取交叉点
            a_offspring_chromosome = []
            a_offspring_chromosome.extend(pair[0].chromosome[:cross_point])
            a_offspring_chromosome.extend(pair[1].chromosome[cross_point:])
            
            # 判断染色体是否合法
            if not self.evaluator.if_valid_chromosome(np.array(a_offspring_chromosome)):
                individual = Individual(self.evaluator,fixed_chromosome=a_offspring_chromosome)
                self.offspring.append(individual)
    
    def point_mutation(self):
        for i in range(len(self.offspring)):
            tmp = random.uniform(0,1)  
            if tmp < p_point_mutation:  # 突变概率
                mutation = random.randint(0, evaluator.gene_num-1)  # 随机选取突变点
                self.offspring[i].chromosome[mutation] = bool(1-self.offspring[i].chromosome[mutation])  # 点突变
                
                

In [31]:
# 常数定义
weight_MAX = 1000  # 背包的最大容量
weights = [80,82,85,70,72,70,66,50,55,25,50,55,40,48,50,32,22,60,30,32,40,38,35,32,25,28,30,22,25,30,45,30,60,50,20,65,20,25,30,10,20,25,15,10,10,10,4,4,2,1]  # 物品所占容量
values= [220,208,198,192,180,180,165,162,160,158,155,130,125,122,120,118,115,110,105,101,100,100,98,96,95,90,88,82,80,77,75,73,72,70,69,66,65,63,60,58,56,50,30,20,15,10,8,5,3,1]  # 物品价值
gene_num=50  # 染色体内基因的个数，在本例中即为物品个数
p_point_mutation = 0.3  # 突变频率
iter_times = 100  # 种群迭代次数
population_size = 100  # 种群内包含多少个个体
offspring_size = 30  # 每次产生多少后代

start = time.time()
# 初始化
evaluator = Evaluator(gene_num,values,weights,weight_MAX)  # 评价类
population = Population(population_size,evaluator)  # 种群

for i in range(iter_times):
    selector_operator = SelectionOperator(population,offspring_size)  # 选择算子
    parents = selector_operator.roulette_wheel()  # 使用轮盘赌算法选择父代

    variation_operator = VariationOperator(evaluator,parents,p_point_mutation)  # 变异算子（交叉、突变）
    offspring = variation_operator.get_offspring()  # 经交叉和突变后获得子代
    
    population.evolve(offspring)  # 子代加入种群，种群内进行淘汰，维持原始个体数

# 结果展示
best_individual = max(population.best_individual_dict, key=population.best_individual_dict.get)
print('Time used: ', time.time()-start, 's')
print('best chromosome: ',best_individual.chromosome)
print('best fitness value: ',best_individual.fitness_value)

Time used:  2.2092950344085693 s
best chromosome:  [1 1 0 1 1 1 1 1 0 1 1 0 1 0 0 1 0 0 0 0 1 0 1 1 1 1 1 0 1 0 1 1 1 0 1 0 1
 1 0 1 0 0 1 0 0 0 0 0 1 0]
best fitness value:  3018


In [32]:
population.best_individual_dict

{<__main__.Individual at 0x1161d2470>: 2807,
 <__main__.Individual at 0x1161f04a8>: 2841,
 <__main__.Individual at 0x1161d22e8>: 2889,
 <__main__.Individual at 0x1161d2d68>: 2953,
 <__main__.Individual at 0x1161e5438>: 3005,
 <__main__.Individual at 0x1161e5a20>: 3016,
 <__main__.Individual at 0x1161f07b8>: 3018}